In [63]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import urllib.request
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from konlpy.tag import Mecab
from tqdm import tqdm
from torchtext import data
from torchtext.data import Iterator

mecab = Mecab()

location = '/Users/kimjonghun/Desktop/3-2학기/캡스톤/open/'

train_csv = pd.read_csv(location + 'train.csv')
test_csv = pd.read_csv(location + 'test.csv')

print(train_csv.shape)
print(train_csv.isnull().sum())
train_csv.dropna(inplace = True)
print(train_csv.shape)
print(test_csv.shape)


(16986, 6)
id          0
img_path    0
overview    0
cat1        0
cat2        0
cat3        0
dtype: int64
(16986, 6)
(7280, 3)


In [64]:
train_csv

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [65]:
train_csv['overview'] = train_csv.overview.apply(mecab.nouns)

In [66]:
df_drop = train_csv[['overview', 'cat3']]

In [67]:
#cat3 원핫인코딩
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn import preprocessing

#Step1: 모든 문자를 숫자형으로 변환합니다.
encoder = preprocessing.LabelEncoder()
encoder.fit(df_drop['cat3'])
cat3 = encoder.transform(df_drop['cat3'])

#Step2: 2차원 데이터로 변환합니다.
cat3 = cat3.reshape(-1, 1)

#Step3: One-Hot Encoding 적용합니다.
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(cat3)
one_hot_cat3 = one_hot_encoder.transform(cat3)
print(one_hot_cat3.toarray())
print(one_hot_cat3.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(16986, 128)


In [74]:
#overview 정수인코딩
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_csv['overview'])
encoded_overview = tokenizer.texts_to_sequences(train_csv['overview'])


In [75]:
max_len = max(len(item) for item in encoded_overview)
print('최대 길이 :',max_len)

최대 길이 : 1475


In [101]:
max_len = 1475
encoded_overview = pad_sequences(encoded_overview, maxlen=max_len)

In [102]:
encoded_overview.shape

(16986, 1475)

In [104]:
#one_hot_cat3 = one_hot_cat3.toarray()

In [109]:
from sklearn.model_selection import train_test_split

data = encoded_overview
target = one_hot_cat3

# train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=34)

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 45000
embedding_dim = 300 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(128, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val), callbacks = [es, mc])

Epoch 1/20


2022-11-14 23:27:14.114674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


425/425 [==============================] - ETA: 0s - loss: 2.7971 - accuracy: 0.4161

2022-11-14 23:28:23.978470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


425/425 [==============================] - 75s 175ms/step - loss: 2.7971 - accuracy: 0.4161 - val_loss: 1.8082 - val_accuracy: 0.5931
Epoch 2/20
425/425 [==============================] - 70s 164ms/step - loss: 1.5948 - accuracy: 0.6306 - val_loss: 1.1108 - val_accuracy: 0.7466
Epoch 3/20
425/425 [==============================] - 69s 163ms/step - loss: 1.0257 - accuracy: 0.7482 - val_loss: 0.7009 - val_accuracy: 0.8424
Epoch 4/20
425/425 [==============================] - 68s 161ms/step - loss: 0.5826 - accuracy: 0.8517 - val_loss: 0.4662 - val_accuracy: 0.8988
Epoch 5/20
425/425 [==============================] - 73s 171ms/step - loss: 0.3004 - accuracy: 0.9220 - val_loss: 0.3783 - val_accuracy: 0.9252
Epoch 6/20
425/425 [==============================] - 79s 185ms/step - loss: 0.1477 - accuracy: 0.9652 - val_loss: 0.3483 - val_accuracy: 0.9329
Epoch 7/20
425/425 [==============================] - 76s 178ms/step - loss: 0.0787 - accuracy: 0.9841 - val_loss: 0.3466 - val_accuracy: 0.9

In [112]:
#model.save(location + 'keras_Text_Cnn1d')

INFO:tensorflow:Assets written to: /Users/kimjonghun/Desktop/3-2학기/캡스톤/open/keras_Text_Cnn1d/assets


INFO:tensorflow:Assets written to: /Users/kimjonghun/Desktop/3-2학기/캡스톤/open/keras_Text_Cnn1d/assets


In [114]:
from tensorflow import keras 
model = keras.models.load_model(location + 'keras_Text_Cnn1d')

In [115]:
test_csv

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [117]:
test_csv['overview'] = test_csv.overview.apply(mecab.nouns)

In [118]:
#test_ overview 정수인코딩
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_csv['overview'])
encoded_overview_test = tokenizer.texts_to_sequences(test_csv['overview'])


In [120]:
encoded_overview_test[0]

[238, 654, 36, 218, 6, 4520]

In [121]:
max_len = max(len(item) for item in encoded_overview_test)
print('최대 길이 :',max_len)

최대 길이 : 1299


In [122]:
max_len = 1299
encoded_overview_test = pad_sequences(encoded_overview_test, maxlen=max_len)

In [125]:
prediction = model.predict(encoded_overview_test)

228/228 [==============================] - 8s 34ms/step
